### 이웃기반 협업필터링 구현
- 이웃기반 협업필터링에서 유사 계산 : 자카드, 피어슨, 코사인

In [1]:
import os
import pandas as pd
import numpy as np
from math import sqrt
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
cd '/content/drive/MyDrive/Colab Notebooks/busan_ai/module10_recommend sys'

[Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/busan_ai/module10_recommend sys'
/content


In [3]:
df= pd.read_csv('movielens/ratings.csv', encoding='utf-8') # 한글깨짐 방지하기위해 utf-8
train, test = train_test_split(df, test_size=0.2, random_state = 24)

FileNotFoundError: ignored

In [ ]:
sparse_matrix = train.groupby('movieId').apply(lambda x: pd.Series(x['rating'].values, index = x['userId'])).unstack()

In [ ]:
sparse_matrix = sparse_matrix.fillna(0)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def cossim_matrix(a,b):
    cossim_values = cosine_similarity(a.values, b.values)
    cossim_df = pd.DataFrame(data=cossim_values, columns = a.index.values, index = a.index)

    return cossim_df

### 이웃기반 협업필터링 추천점수 계산
아이템 기반

In [ ]:
item_cossim_df = cossim_matrix(sparse_matrix, sparse_matrix)

In [ ]:
userid_grouped = train.groupby('userId')
item_prediction_result_df = pd.DataFrame(index = list(userid_grouped.indices.keys()), columns = sparse_matrix.index)

In [ ]:
for userid, group in tqdm(userid_grouped):
    user_sim = item_cossim_df.loc[group['movieId']]

    user_rating = group['rating']
    sim_sum = user_sim.sum(axis = 0)
    pred_ratings = np.matmul(user_sim.T.to_numpy(), user_rating) / (sim_sum + 1)
    item_prediction_result_df.loc[userid] = pred_ratings

In [ ]:
print(item_prediction_result_df)

In [ ]:
user_sparse_matrix = sparse_matrix.T

In [ ]:
user_cossim_df = cossim_matrix(user_sparse_matrix, user_sparse_matrix)

In [ ]:
user_cossim_df

In [ ]:
movieid_grouped = train.groupby('movieId')
userId_prediction_result_df = pd.DataFrame(index = list(movieid_grouped.indices.keys()), columns = sparse_matrix.index)

In [ ]:
for movieId, group in tqdm(movieid_grouped):
    user_sim = user_cossim_df.loc[group['userId']]
    user_rating = group['rating']
    sim_sum = user_sim.sum(axis = 0)
    pred_ratings = np.matmul(user_sim.T.to_numpy(), user_rating) / (sim_sum + 1)
    userId_prediction_result_df.loc[movieId] = pred_ratings

In [ ]:
print(userId_prediction_result_df[6])

### RMSE로 추천시스템 성능 평가

In [ ]:
def evaluation(test_df, prediction_result_df):
    groups_with_movie_ids = test_df.groupby(by='movieId')
    groups_with_user_ids = test_df.groupby(by='userId')
    intersection_movie_ids = sorted(list(set(prediction_result_df.columns) - set(groups_with_movie_ids.indices.keys()))) 
    intersection_user_ids = sorted(list(set(prediction_result_df.columns) - set(groups_with_user_ids.indices.keys()))) 
    
    compressed_prediction_df = prediction_result_df.loc[intersection_user_ids][intersection_movie_ids]
    
    grouped = test_df.groupby(by="userId")
    result_df = pd.DataFrame(columns=['rmse'])

    for userId, group in tqdm(grouped):
      if userId in intersection_user_ids:
            pred_ratings = compressed_prediction_df.loc[userId][compressed_prediction_df.loc[userId].index.intersection(list(group['movieId'].values))]
            pred_ratings = pred_ratings.to_frame(name='rating').reset_index().rename(columns={'index':'movieId','rating':'pred_rating'})
            actual_ratings = group[['rating', 'movieId']].rename(columns={'rating':'actual_rating'})
            final_df = pd.merge(actual_ratings, pred_ratings, how='inner', on=['movieId'])
            final_df = final_df.round(4) # 반올림
          # if not final_df.empty:
          #     rmse = sqrt(mean_squared_error(final_df['rating_actual'], final_df['rating_pred']))
          #     result_df.loc[userId] = rmse
          #     # print(userId, rmse)
    return final_df

In [ ]:
evaluation(test, userId_prediction_result_df)